# Setup:
The code sets up essential configurations and clients to interact with AWS services and the Amazon Bedrock agent runtime. It imports required modules, a custom observability module, and fetches various configuration values.

In [1]:
import boto3
import time
import json

# Custom Module:
from observability import BedrockLogs

# Import configuration values
from config import (
    REGION, FIREHOSE_NAME, CRAWLER_NAME, MODEL_ARN, AGENT_ID, AGENT_ALIAS_ID, 
    SESSION_ID, ENABLE_TRACE, END_SESSION, AGENT_CONFIG, EXPERIMENT_ID, 
    CUSTOM_TAG, GUARDRAIL_ID, GUARDRAIL_VERSION, MAX_TOKENS, TEMPERATURE, TOP_P
)

bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

# Initialize BedrockLogs for observability using Amazon Firehose:
# bedrock_logs = BedrockLogs(delivery_stream_name=FIREHOSE_NAME, 
#                            experiment_id=EXPERIMENT_ID, 
#                            feature_name='Agent')

# Initialize BedrockLogs for observability using local mode for troubleshooting:
bedrock_logs = BedrockLogs(delivery_stream_name='local', feedback_variables=True, 
                           feature_name='Agent')

# Setup Observability:

The `invoke_agent` function calls an agent API, processes the response to extract the agent's answer and trace data, handles exceptions, and returns relevant information. It is decorated to enable logging or monitoring of the function call.

In [2]:
# invoke the agent API
@bedrock_logs.watch(call_type='agent-in-prod')
def invoke_agent(query_to_agent, agent_id, agent_alias_id,
                 session_id, enableTrace, endSession, agent_config=None):
    try:
        start_time = time.time()
        time_to_first_token = None
        time_at_first_token = None
        
        agentResponse = bedrock_agent_runtime_client.invoke_agent(
            inputText=query_to_agent,
            agentId=agent_id,
            agentAliasId=agent_alias_id,
            sessionId=session_id,
            enableTrace=enableTrace,
            endSession=endSession
        )

        event_stream = agentResponse['completion']
        agent_answer = None
        end_event_received = False
        trace_data = []

        for event in event_stream:
            if 'chunk' in event:
                if time_to_first_token is None:
                    time_at_first_token = time.time()
                    time_to_first_token = time_at_first_token - start_time
                data = event['chunk']['bytes']
                agent_answer = data.decode('utf8')
                end_event_received = True
            elif 'trace' in event:
                trace = event['trace']
                trace['start_trace_time'] = time.time()
                trace_data.append(trace)
            else:
                raise Exception("Unexpected event.", event)

        if not end_event_received:
            raise Exception("End event not received.")
            
        agentResponse['ResponseMetadata']['time_to_first_token'] = time_to_first_token
        agentResponse['ResponseMetadata']['time_to_last_token'] = time.time() - time_at_first_token

    except Exception as e:
        raise e
    
    # the following will be returned as a tuple datatype:
    return agentResponse['ResponseMetadata'], agent_answer, trace_data

# Test the observability

Here we pass a question to the invoke_agent custom function and expect that `function arguments`, `model response`, and `traces` will be logged to the configured Amazon S3 bucket.

In [3]:
%%time
QUESTION = "<enter-your-question-here>"

results, log, run_id, observation_id = invoke_agent(
    QUESTION, 
    AGENT_ID, 
    AGENT_ALIAS_ID, 
    SESSION_ID, 
    ENABLE_TRACE, 
    END_SESSION
)

Logs in local mode:
CPU times: user 17.3 ms, sys: 8.59 ms, total: 25.9 ms
Wall time: 25.3 s


# END